In [1]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')


1--- 2.288970470428467 seconds ---
2--- 0.15350008010864258 seconds ---
3--- 4.514971733093262 seconds ---
4--- 0.960146427154541 seconds ---
5--- 0.08275103569030762 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
DECILE,,,,,,,
0.0,0.853193,0.986338,0.957256,1.062841,1.083444,1.112252,0.875452
1.0,0.914240,1.149675,0.997889,1.127245,1.049022,1.037948,0.908440
2.0,0.999011,1.256870,1.110506,1.080057,1.165654,1.051439,0.916550
3.0,0.999760,1.243147,1.060718,1.127975,1.032943,1.115772,0.970901
4.0,0.967414,1.244716,1.025827,1.098230,1.219971,1.101547,0.920304
5.0,1.003889,1.349811,1.019209,1.081615,1.034600,1.127087,0.943066
6.0,1.074467,1.261159,1.170995,1.143669,1.037079,1.092252,0.999023
7.0,1.052130,1.279966,1.203802,1.087405,1.096980,1.262767,0.985489
8.0,1.112460,1.406287,1.193011,1.115821,1.021640,1.265122,0.937511
